In [4]:
import tensorflow as tf

In [5]:
#这次，我们从1个数据点的6个样本开始
x_input_data = tf.random_normal([6], mean=-1, stddev=4)

#请注意，FIFO队列的容量还是3
q = tf.FIFOQueue(capacity=3, dtypes=tf.float32)

#检查这种情况发生了什么：
#我们将在每次“x_input_data”实际计算时打印一条消息
#用于“enqueue_many”操作
x_input_data = tf.Print(x_input_data, data=[x_input_data],
                        message='Raw inputs data generated:',summarize=6)
enqueue_op = q.enqueue_many(x_input_data)

#为了利用多线程，我们创建一个“QueueRunner”
#将处理主线程外的“enqueue_op”
#我们在这里不需要太多的并行性，所以我们只使用1个线程
numberOfThreads = 1
qr = tf.train.QueueRunner(q, [enqueue_op] * numberOfThreads)
#不要忘记将“QueueRunner”添加到QUEUE_RUNNERS集合中
tf.train.add_queue_runner(qr)

input = q.dequeue()
input = tf.Print(input, data=[q.size(), input], message='Nb elements left, input:')

#fake graph :START
y = input + 1
#fake graph :END

with tf.Session() as sess:
    #但是现在我们构建我们的协调器来协调我们的主线程的子线程
    coord = tf.train.Coordinator()
    #小心，如果你在runnig之前没有启动所有的队列
    #主线程将等待他们启动，你将再次挂起
    #这个帮助器启动tf.GraphKeys.QUEUE_RUNNERS中的所有队列
    threads = tf.train.start_queue_runners(coord=coord)
    
    #QueueRunner会自动在自己的线程中异步调用入队操作，
    #确保队列总是完全不再挂起主进程，不再等待GPU
    sess.run(y)
    sess.run(y) 
    sess.run(y)
    sess.run(y)
    sess.run(y)
    sess.run(y)
    sess.run(y)
    sess.run(y)
    sess.run(y)
    sess.run(y)

    #我们要求我们的子线程停止...
    coord.request_stop()
    #然后我们等待他们这样做，然后释放主线程
    coord.join(threads)